# Estimating Relative Orientation using Essential Matrix 

In this exercise your tasks are to estimate the essential matrix given two images and estimate the relative orientation based on the estimated essential matrix. Two images of the checkerboard cube and the intrinsic matrix (K) are given.

## 4.4.0 Data: 3D Cube with checkerboard pattern

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ex4_4 as ex

K = np.array(
 [[-1.23543537e+03,  1.08226965e+01,  2.82869653e+02],
  [-0.00000000e+00, -1.23745635e+03,  4.69737272e+02],
  [+0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

# load the input image
fig = plt.figure()
fig.add_subplot(1,2,1)
I1 = plt.imread('data/checkerboard_cube/cube0.jpg')
plt.imshow(I1, cmap = 'gray')

fig.add_subplot(1,2,2)
I2 = plt.imread('data/checkerboard_cube/cube1.jpg')
plt.imshow(I2, cmap = 'gray')
plt.show()

## 4.4.1 Select enough correspondences to estimate the Essential Matrix [5.0]

In [ ]:
import matplotlib
matplotlib.use('TkAgg')

num_corr = None  # please sepecify the needed points here
new_order = [1, 0]  # Switch x & y axis st normal from image plane is towards camera origin

print("\n Please click ", num_corr, " points in the first image...\n")

# Get observed correspondences from user
I1 = plt.imread('data/checkerboard_cube/cube0.jpg')
plt.imshow(I1, cmap='gray')
pts1 = np.array(plt.ginput(num_corr))
pts1 = pts1[:, new_order]

pts1 = ex.gen_homogeneous_pts(pts1)
print("\n points in the first image:\n ", pts1)

print("\n Please click ", num_corr, " corresponding points in the second image...\n")

I2 = plt.imread('data/checkerboard_cube/cube1.jpg')
plt.imshow(I2, cmap='gray')
pts2 = np.array(plt.ginput(num_corr))
pts2 = pts2[:, new_order]
    
pts2 = ex.gen_homogeneous_pts(pts2)    
print("\n points in the second image:\n ", pts1)

# compute essential matrix from point pairs
E = ex.E_from_point_pairs(pts1, pts2, K)
print('\n Essential Matrix:\n', E)

## 4.4.2 Estimate the relative orientation based on the Essential matrix [5.0]

In [ ]:
# given E compute relative orientation",
Z = np.array([[ 0, 1, 0],
              [-1, 0, 0],
              [ 0, 0, 0]])
W = np.array([[0, -1, 0],
              [1,  0, 0],
              [0,  0, 1]])

W_CHECK = [W, W.T]
Z_CHECK = [Z, Z.T]

for w in W_CHECK:
    for z in Z_CHECK:
        R, Sb = ex.relative_orientation_from_E(E, z, w)
        b = np.array([-Sb[1, 2], Sb[0, 2], -Sb[0, 1]])

        # Triangulate points in coord system of 1st cam
        X0 = ex.triangulate_points(pts1, pts2, K, R, b)

        # Triangulated points in coord system of 2nd cam
        X1 = np.dot(R.T, X0 - np.expand_dims(b, 1))

        if ex.point_in_front_of_cam(X0) and ex.point_in_front_of_cam(X1):
            print('Final Rotation:\n', R)
            R_final = R
            print('Final Baseline:\n', b)
            b_final = b